##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Crie um pipeline TFX usando templates com orquestrador local

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/template_local"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/template_local.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Ver fonte no GitHub</a>
</td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

## Introdução

Este documento fornecerá instruções para criar um pipeline TensorFlow Extended (TFX) usando *templates* fornecidos com o pacote TFX Python. A maioria das instruções são comandos shell do Linux. Células de código correspondentes do Jupyter Notebook que invocam esses comandos usando `!` são fornecidos.

Você construirá um pipeline usando o [dataset Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) divulgado pela cidade de Chicago. Recomendamos fortemente que você tente construir seu pipeline usando seu próprio dataset, utilizando esse pipeline como referência.

Construiremos um pipeline que roda em ambiente local. Se você estiver interessado em usar o orquestrador Kubeflow no Google Cloud, consulte o [tutorial TFX no Cloud AI Platform Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines).

## Pré-requisitos

- Linux / MacOS
- Python &gt;= 3.5.3

Você pode obter todos os pré-requisitos facilmente [executando este notebook no Google Colab](https://colab.sandbox.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb).


## Etapa 1. Configure seu ambiente.

**Ao longo deste documento, apresentaremos comandos duas vezes. Uma vez como um comando shell pronto para copiar e colar, uma vez como uma célula do notebook jupyter. Se você estiver usando o Colab, basta pular os blocos de script shell e executar as células do notebook.**

Você deve preparar um ambiente de desenvolvimento para construir um pipeline.

Instale o pacote `tfx` python. Recomendamos o uso de `virtualenv` no ambiente local. Você pode usar o seguinte snippet de script de shell para configurar seu ambiente.

```sh
# Create a virtualenv for tfx.
virtualenv -p python3 venv
source venv/bin/activate
# Install python packages.
python -m pip install --upgrade "tfx<2"
```

Se você estiver usando o colab:


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade "tfx<2"

OBSERVAÇÃO: Pode haver alguns erros durante a instalação do pacote. Por exemplo,

> ERROR: some-package 0.some_version.1 has requirement other-package!=2.0.,&lt;3,&gt;=1.15, but you'll have other-package 2.0.0 which is incompatible.

Por favor, ignore esses erros neste momento.

In [ ]:
# Set `PATH` to include user python binary directory.
HOME=%env HOME
PATH=%env PATH
%env PATH={PATH}:{HOME}/.local/bin

Vamos verificar a versão do TFX.

```bash
python -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"
```

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

E está feito. Estamos prontos para criar um pipeline.

## Etapa 2. Copie o template predefinido para o diretório do seu projeto.

Nesta etapa, criaremos um diretório e arquivos de projeto de pipeline funcionais, copiando arquivos adicionais de um template predefinido.

Você pode dar um nome diferente ao seu pipeline alterando `PIPELINE_NAME` abaixo. Este também se tornará o nome do diretório do projeto onde seus arquivos serão colocados.

```bash
export PIPELINE_NAME="my_pipeline"
export PROJECT_DIR=~/tfx/${PIPELINE_NAME}
```

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Create a project directory under Colab content directory.
PROJECT_DIR=os.path.join(os.sep,"content",PIPELINE_NAME)

O TFX inclui o template `taxi` com o pacote TFX python. Se você está planejando resolver um problema de previsão pontual, incluindo classificação e regressão, este template pode ser usado como ponto de partida.

O comando CLI `tfx template copy` copia arquivos de template predefinidos no diretório do projeto.

```sh
tfx template copy \
   --pipeline_name="${PIPELINE_NAME}" \
   --destination_path="${PROJECT_DIR}" \
   --model=taxi
```

In [ ]:
!tfx template copy \
  --pipeline_name={PIPELINE_NAME} \
  --destination_path={PROJECT_DIR} \
  --model=taxi

Altere o contexto do diretório de trabalho neste notebook para o diretório do projeto.

```bash
cd ${PROJECT_DIR}
```

In [ ]:
%cd {PROJECT_DIR}

## Etapa 3. Navegue pelos arquivos-fonte copiados.

O template TFX fornece arquivos de scaffold básicos para construir um pipeline, incluindo código-fonte Python, dados de amostra e Jupyter Notebooks para analisar a saída do pipeline. O template `taxi` usa o mesmo dataset do *Chicago Taxi* e modelo de ML do [Tutorial Airflow](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop).

No Google Colab, você pode navegar pelos arquivos clicando no ícone de uma pasta à esquerda. Os arquivos devem ser copiados no diretório do projeto, cujo nome é `my_pipeline` neste caso. Você pode clicar nos nomes dos diretórios para ver o conteúdo do diretório e clicar duas vezes nos nomes dos arquivos para abri-los.

Aqui está uma breve introdução a cada um dos arquivos Python.

- `pipeline` - Este diretório contém a definição do pipeline
    - `configs.py` — define constantes comuns para executores de pipeline
    - `pipeline.py` — define componentes TFX e um pipeline
- `models` - Este diretório contém definições de modelo de ML.
    - `features.py`, `features_test.py` — define características para o modelo
    - `preprocessing.py`, `preprocessing_test.py` — define jobs de pré-processamento usando `tf::Transform`
    - `estimator` — este diretório contém um modelo baseado em Estimator.
        - `constants.py` — define constantes do modelo
        - `model.py`, `model_test.py` — define o modelo DNN usando o estimador TF
    - `keras` — este diretório contém um modelo baseado em Keras.
        - `constants.py` — define constantes do modelo
        - `model.py`, `model_test.py` — define o modelo DNN usando Keras
- `local_runner.py`, `kubeflow_runner.py` — define executores para cada mecanismo de orquestração


Você poderá perceber que existem alguns arquivos com `_test.py` em seus nomes. Estes são testes unitários do pipeline e é recomendado adicionar mais testes unitários à medida que você for implementando seus próprios pipelines. Você pode executar testes de unidade fornecendo o nome do módulo dos arquivos de teste com o sinalizador `-m`. Geralmente você pode obter um nome de módulo excluindo a extensão `.py` e substituindo `/` por `.`. Por exemplo:

```bash
python -m models.features_test
```

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## Etapa 4. Execute seu primeiro pipeline TFX

Você pode criar um pipeline usando o comando `pipeline create`.

```bash
tfx pipeline create --engine=local --pipeline_path=local_runner.py
```


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

Em seguida, você pode executar o pipeline criado usando o comando `run create`.

```sh
tfx run create --engine=local --pipeline_name="${PIPELINE_NAME}"
```

In [ ]:
!tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

Se for bem-sucedido, você verá `Component CsvExampleGen is finished.` Ao copiar o template, apenas um componente, o CsvExampleGen, será incluído no pipeline.

## Etapa 5. Adicione componentes para validação de dados.

Nesta etapa, você adicionará componentes para validação de dados, incluindo `StatisticsGen`, `SchemaGen` e `ExampleValidator`. Se você estiver interessado na validação de dados, consulte [Introdução ao Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

Modificaremos a definição do pipeline copiado em `pipeline/pipeline.py`. Se você estiver trabalhando em seu ambiente local, use seu editor favorito para editar o arquivo. Se você estiver trabalhando no Google Colab,

> **Clique no ícone da pasta à esquerda para abrir a visualização `Files`**.

> **Clique em `my_pipeline` para abrir o diretório, depois clique no diretório `pipeline` para abrir e dê um duplo-clique em `pipeline.py` para abrir o arquivo**.

> Encontre e descomente as 3 linhas que adicionam `StatisticsGen`, `SchemaGen` e `ExampleValidator` ao pipeline. (Dica: encontre comentários contendo `TODO(step 5):`).

> Sua alteração será salva automaticamente em alguns segundos. Certifique-se de que a marca `*` na frente de `pipeline.py` tenha desaparecido no título da aba. **Não há botão para salvar ou atalho para o editor de arquivos no Colab. Os arquivos Python no editor de arquivos podem ser salvos no ambiente de execução, mesmo no modo `playground`.**

Agora você precisa atualizar o pipeline existente com a definição de pipeline modificada. Use o comando `tfx pipeline update` para atualizar seu pipeline, seguido pelo comando `tfx run create` para criar uma nova execução de seu pipeline atualizado.

```sh
# Update the pipeline
tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```


In [ ]:
# Update the pipeline
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

Você deverá conseguir ver o log de saída dos componentes adicionados. Nosso pipeline cria artefatos de saída no diretório `tfx_pipeline_output/my_pipeline`.

## Etapa 6. Adicione componentes para treinamento.

Nesta etapa, você adicionará componentes para treinamento e validação de modelo, incluindo `Transform`, `Trainer`, `Resolver`, `Evaluator` e `Pusher`.

> **Abra `pipeline/pipeline.py`**. Encontre e descomente 5 linhas que adicionam `Transform`, `Trainer`, `Resolver`, `Evaluator` e `Pusher` ao pipeline. (Dica: encontre `TODO(step 6):`)

Como fez antes, agora você precisa atualizar o pipeline existente com a definição de pipeline modificada. As instruções são as mesmas da Etapa 5. Atualize o pipeline usando `tfx pipeline update` e crie uma execução usando `tfx run create`.

```sh
tfx pipeline update --engine=local --pipeline_path=local_runner.py
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

Quando esta execução for concluída com sucesso, você terá criado e executado seu primeiro pipeline TFX usando o orquestrador local!

**OBSERVAÇÃO:** Você deve ter percebido que toda vez que criamos uma execução de pipeline, cada componente é executado repetidamente, mesmo que a entrada e os parâmetros não tenham sido alterados. É uma perda de tempo e recursos, e você pode pular essas execuções com o cache do pipeline. Você pode ativar o cache especificando `enable_cache=True` para o objeto `Pipeline` em `pipeline.py`.


## Etapa 7. (*Opcional*) Experimente BigQueryExampleGen.

[BigQuery] é um data warehouse em nuvem sem servidor, altamente escalonável e econômico. O BigQuery pode ser usado como fonte para exemplos de treinamento no TFX. Nesta etapa, adicionaremos `BigQueryExampleGen` ao pipeline.

Você precisa de uma conta [do Google Cloud Platform](https://cloud.google.com/gcp/getting-started) para usar o BigQuery. Prepare um projeto GCP.

Faça login no seu projeto usando a biblioteca de autenticação colab ou o utilitário `gcloud`.

```sh
# You need `gcloud` tool to login in local shell environment.
gcloud auth login
```

In [ ]:
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')


Você deve especificar o nome do seu projeto GCP para acessar os recursos do BigQuery usando o TFX. Defina a variável de ambiente `GOOGLE_CLOUD_PROJECT` como o nome do seu projeto.

```sh
export GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE
```

In [ ]:
# Set your project name below.
# WARNING! ENTER your project name before running this cell.
%env GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE

> **Abra `pipeline/pipeline.py`**. Comente `CsvExampleGen` e remova o comentário da linha que cria uma instância de `BigQueryExampleGen`. Você também precisa descomentar o argumento `query` da função `create_pipeline`.

Precisamos especificar qual projeto GCP usar para o BigQuery novamente, e isso é feito definindo `--project` em `beam_pipeline_args` ao criar um pipeline.

> **Abra `pipeline/configs.py`**. Remova o comentário da definição de `BIG_QUERY__WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` e `BIG_QUERY_QUERY`. Você deve substituir o ID do projeto e o valor da região neste arquivo pelos valores corretos para o seu projeto do GCP.

> **Abra `local_runner.py`**. Remova o comentário de dois argumentos, `query` e `beam_pipeline_args`, para o método create_pipeline().

Agora o pipeline está pronto para usar o BigQuery como fonte de exemplos. Atualize o pipeline e crie uma execução como fizemos nas etapas 5 e 6.

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

## O que vem depois: fornecer SEUS próprios dados ao pipeline.

Fizemos um pipeline para um modelo usando o dataset Chicago Taxi. Agora é hora de colocar seus dados no pipeline.

Seus dados podem ser armazenados em qualquer lugar que seu pipeline possa acessar, incluindo GCS ou BigQuery. Você precisará modificar a definição do pipeline para acessar seus dados.

1. Se seus dados estiverem armazenados em arquivos, modifique `DATA_PATH` em `kubeflow_runner.py` ou `local_runner.py` e defina-o como o local de seus arquivos. Se seus dados estiverem armazenados no BigQuery, modifique `BIG_QUERY_QUERY` em `pipeline/configs.py` para consultar seus dados corretamente.
2. Adicione características em `models/features.py`.
3. Modifique `models/preprocessing.py` para [transformar os dados de entrada para treinamento](https://www.tensorflow.org/tfx/guide/transform).
4. Modifique `models/keras/model.py` e `models/keras/constants.py` para [descrever seu modelo de ML](https://www.tensorflow.org/tfx/guide/trainer).
    - Você também pode usar um modelo baseado em estimador. Altere a constante `RUN_FN` para `models.estimator.model.run_fn` em `pipeline/configs.py`.

Consulte o [Guia do componente Trainer](https://www.tensorflow.org/tfx/guide/trainer) para mais informações.